In [1]:
%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import itertools

import os
import requests
import bs4
import time as tm
import http.client

from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import RFE
import sklearn.linear_model

from xgboost import XGBRegressor
from matplotlib import pyplot
from xgboost import plot_importance

from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf


%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

from bayes_opt import BayesianOptimization
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

##### for python 3: conda install -c anaconda py-xgboost 
##### for python 2: anaconda search -t conda xgboost => conda install -c mndrake xgboost

##### get train data (may 2014 - april 2016) taxi dataframe; test and verify data (may, june 2016) and create join frame - taxi_prediction

In [2]:
train_file = "./data/reg_bin_stat_2years.pkl"
verify_file = "./data/verif_bin_stat_5_6_2016.pkl"

In [3]:
with open(train_file, "rb") as f:
    data_2year = pickle.load(f)

with open(verify_file, "rb") as f:
    data_2month = pickle.load(f)    

In [4]:
time = pd.date_range('2014 May 1 00:00:00', periods = data_2year.shape[1] - 1, freq = 'h')
taxi = pd.DataFrame(data_2year.T[1:, :], index = time, columns=[data_2year.T[0, :].astype(int)])

prediction_time = pd.date_range('2016 May 1 00:00:00', '2016 June 30 23:00:00', freq = 'h')
#taxi_prediction = pd.concat([taxi, pd.DataFrame(np.ones((prediction_time.shape[0], taxi.shape[1])), 
#                                                index = prediction_time, columns=[data_2year.T[0, :].astype(int)])], axis=0)
taxi_prediction = pd.concat([taxi, pd.DataFrame(np.array([[None]*taxi.shape[1]]*prediction_time.shape[0]), 
                                                index = prediction_time, columns=[data_2year.T[0, :].astype(int)])], axis=0)

verify_time = pd.date_range('2016 May 1 00:00:00', periods = data_2month.shape[1] - 1, freq = 'h')
verify_taxi = pd.DataFrame(data_2month.T[1:, :], index = verify_time, columns=[data_2month.T[0, :].astype(int)])

##### correspondence map num_of_column => num_of_region

In [5]:
region_map = dict(zip(list(range(102)), taxi.columns.values))

In [6]:
taxi.head()

In [7]:
verify_taxi.head()

##### this week we are going to find appropriate features for linear regression - one of such "good" feature that helps us to get high prediction result may we daily and hourly weather in NY. Code below parse weather site to get temperature, humidity and whethere it was snow rain or fog
##### https://www.wunderground.com/history/airport/KJFK/2014/1/7/DailyHistory.html?req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=

In [ ]:
def prc(url):
    t = requests.get(url)
    t.encoding = 'UTF-8'
    return bs4.BeautifulSoup(t.text, 'lxml', from_encoding=t.encoding)

In [ ]:
day_t = pd.date_range('2014 Jan 1 00:00:00', periods = 1096, freq = 'd')
daily_temp = pd.DataFrame(index = [day_t], columns=["daily_temp"])
hour_weather = pd.DataFrame(columns=["time", "daily_temp", "windchill", "humidity", "events"])

In [ ]:
days = {2014:[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], 
        2015:[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
        2016:[31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]}

In [ ]:
d_temp = []
for year in [2014, 2015, 2016]:
    for month in range(1, 13):
        for day in range(1, days[year][month-1] + 1):

            url = "https://www.wunderground.com/history/airport/KJFK/%d/%d/%d/DailyHistory.html?req_city=&req_state=\
                   &req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=" % (year, month, day)
            try:
                parser = prc(url)
                print(url)
                review = parser.find_all('span', attrs={'class':'wx-value'})
                d_temp.append(int(review[0].text))
                
                review_1 = parser.find_all('table', attrs={'class':'obs-table responsive'})[0].find_all('tr', attrs={'class':'no-metars'})
                #print(len(review_1))
                for tr in review_1:
                    timeAndValues = tr.find_all('td')
                    temp = tr.find_all('span', attrs={'class':'wx-value'})
                    hour_weather.loc[hour_weather.shape[0]] = [pd.to_datetime("%d %d %d" % (year, month, day) + " " + timeAndValues[0].text), 
                                                               temp[0].text, temp[1].text, timeAndValues[4].text, timeAndValues[-2].text]
                
                tm.sleep(1)
            
            except (http.client.HTTPException, http.client.IncompleteRead):
                print(year, month, day)
                continue

daily_temp["daily_temp"] = d_temp  

In [ ]:
#hour_weather.to_csv("./data/NY_hour_weather.csv", columns=["time", "daily_temp", "windchill", "humidity", "events"])
#daily_temp.to_csv("./data/NY_daily_weather.csv", columns=["daily_temp"])

In [6]:
hour_weather = pd.read_csv("H:/Yandex machine learning/finall course coursera/NY_hour_weather.csv", usecols=["time", "daily_temp", "windchill", "humidity", "events"], encoding ="cp1252")
daily_weather = pd.read_csv("H:/Yandex machine learning/finall course coursera/NY_daily_weather.csv", usecols=["daily_temp"], encoding ="cp1252")

hour_weather.events.unique()

#### Events:  0: nothing, 1: snow, 2: fog snow, 3: rain, 4: fog, 5: fog rain, 6: rain snow, 7: rain thunderstorm, 8: rain snow thunderstorm", 9: thunderstorm

In [7]:
hour_weather.events = hour_weather.events.replace(hour_weather.events.unique(), range(10))
hour_weather.humidity = hour_weather.humidity.map(lambda x: x[:2])
hour_weather.time = pd.DatetimeIndex(hour_weather.time).map(lambda x: x.replace(minute=0, second=0))
hour_weather.drop_duplicates(subset=["time"], inplace=True)
hour_weather.set_index('time', inplace=True)
hour_weather.shape

##### we parse weather for full three years - 2014, 2015, 2016. Select period from may 2014 till june 2016 

In [8]:
""" from [2877:20386] """

test_weather = hour_weather.iloc[2877:20386]
print(test_weather.shape)
print(test_weather.index.unique().shape)
test_weather.tail()

In [9]:
prediction_weather = hour_weather.iloc[20386:21850]
prediction_weather.tail()

##### Create regression features - according to week task it will be: <br><br> 1) fourier components - the experience of last week showed - it is enought to take first 70 terms in fourier serie summation with weekly period (T = 168) and first 20 terms in serie with annual period (T = 365*24) - for all 102 regions <br><br> 2)  values from the past - number of trips 1 , 2, 3, 4, 5, 6 hours before; 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 days before <br><br> 3) cumulative trips sum for last half day, last day, last week, last four weeks <br><br> 4) indicies corresponds to hour, day of week, month, week-day/weekend <br><br> 5) temperature, humidity and categorical variable - ( 0: nothing, 1: snow, 2: fog snow, 3: rain, 4: fog, 5: fog rain, 6: rain snow, 7: rain thunderstorm, 8: rain snow thunderstorm", 9: thunderstorm)  <br><br> 6) also to take into account interaction between features - create binary combinations - pair multiplication of all features exСЃept fourier components and weather features <br><br>  Later we will individualРґy select features for each region using xgboost regressor

##### functions that create dataframe with all features

In [10]:
def shift(data, shift_period, column_name):
    data[column_name] = data["taxi_call_num"].shift(periods=shift_period)    

In [11]:
def cumsum(data, min_period, window, column_name):
    data[column_name] = data["taxi_call_num"].shift(1).rolling(min_periods=min_period, window=window).sum()

In [12]:
def binary_combinations(data, fourier_shape):
    for pair in itertools.combinations(data.columns[fourier_shape:].values, 2):
        data[pair[0] + "_" + pair[1]] = data[pair[0]] * data[pair[1]]    

In [13]:
def feature_constructor(region, regres, dataframe=taxi, weather=test_weather):
    ''' regress : ((period, component_num), ...) '''
    data = pd.DataFrame(dataframe.iloc[:, region].values, index = dataframe.index, columns=["taxi_call_num"])
    time = np.arange(0, dataframe.shape[0])
    
    ''' fourier components - as was shown in previous week - it was enought approx. 15 year and 70 week components '''
    for component in regres:
        for w in range(1, component[1]+1):
            data["sin_w_%d_%d" % (component[0], w)] = np.sin(2*np.pi*w*time/component[0]) 
            data["cos_w_%d_%d" % (component[0], w)] = np.cos(2*np.pi*w*time/component[0])
    data = data.round(3)
    fourier_shape = data.shape[1]
    
    
    d = [("d_before", 24), ("two_d_before", 48), ("three_d_before", 72), ("four_d_before", 96), ("five_d_before", 120), 
         ("six_d_before", 144), ("one_h_before", 1), ("two_h_before", 2), ("three_h_before", 3), ("four_h_before", 4), 
         ("five_h_before", 5), ("six_h_before", 6), ("seven_h_before", 7), ("eight_h_before", 8), ("nine_h_before", 9), 
         ("ten_h_before", 10), ("eleven_h_before", 11), ("twelve_h_before", 12)]
    
    for name, value in d:
        shift(data, shift_period=value, column_name=name)
            
    dd = [("half_d_cumsum", (1, 12)), ("d_cumsum", (1, 24)), ("week_cumsum", (1, 168)), ("four_w_cumsum", (1, 168*4))]
    
    for name, value in dd:
        cumsum(data, min_period=value[0], window=value[1], column_name=name)
        
       
    data["hour"] = data.index.hour
    data["week_day"] = data.index.dayofweek
    data["day"] = data.index.day
    data["month"] = data.index.month
    
    binary_combinations(data, fourier_shape)
    
    data["weekend"] = [1 if (date.weekday == 5 or date.weekday == 6) else 0 for date in data.index]
        
    data = pd.concat([data, weather.iloc[:, [0, 1, 3]]], axis=1)
    data = data.fillna(0.1)
    
    return data, fourier_shape    

##### using xgboost regressor sort features by its "predictive power" (r2 coefficient) and discard non predictive features

In [14]:
def find_best_features(regions):
    ''' find best features using xgboost. for each region take best features with cumsum 99%  '''
    features_good_regions = []
    for reg in regions:
        print("REGION: ", reg, end=" ")
        model = XGBRegressor()
        region, _ = feature_constructor(reg, regres=((168, 70), (8760, 20)), dataframe=taxi, weather=test_weather)
        model.fit(region.iloc[:, 1:], region.taxi_call_num.values)

        significance_list = list(filter(lambda x: x[1] > 0, 
                                        sorted(zip(region.columns[1:], model.feature_importances_), 
                                               key=lambda x: x[1], reverse=True)))
        num_of_features = np.argwhere(np.cumsum([x[1] for x in significance_list]) > 0.95)[0][0]
        features_good_regions.append([x[0] for x in significance_list][:num_of_features])
    return features_good_regions

In [20]:
features = find_best_features(regions=list(range(102)))

##### look at average number of features after features selection

In [23]:
"""  features with interaction """
z = list(map(lambda x: len(x), features))
print("mean amount of features with features interaction: ", sum(z) * 1.0 / len(z))

In [24]:
#with open("./data/best_features_modified_combinations.pickle", 'wb') as handle:
#    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

#os.path.getsize("./data/best_features.pickle")

In [31]:
with open("./data/best_features_modified_combinations.pickle", 'rb') as handle:
    features = pickle.load(handle)

##### for region in first column plot histograme to see what features are more or less important and what are absolutely useless

In [118]:
mod = XGBRegressor()
first, _ = feature_constructor(region=1, regres=((168, 70), (8760, 20)))
mod.fit(first.iloc[:, 1:], first.taxi_call_num.values)

In [119]:
""" plot importance of features with binary interactions between them for region number 1"""
names = first.columns

pyplot.figure(figsize=(first.shape[1]//4, 7))
pyplot.bar(range(len(mod.feature_importances_)), mod.feature_importances_)
pyplot.xticks(np.arange(names.shape[0]), names[1:], rotation=90)
pyplot.axvline(180, color='k', linestyle='dashed', linewidth=1)
pyplot.show()

#double click on image makes it bigger 

#### We will try to get best fit on train data simultaneously trying to achieve good result at test data set using linear regressor ElasticNet() and method based on utilizing decisive trees - xgboost regressor

### Try to figure out - does features combination makes r2 value better of train (may 2014 - april 2016)at test (first week of may 2016) set.    
##### it is possible to do with comment/uncomment binary_combinations() in function feature_constructor()).  use ElasticNet() as a regressor model, plot result as a histogram

In [15]:
def train_linear(region, features_regions, regularizer=None, dataframe=taxi, weather=test_weather):
    data, _ = feature_constructor(region=region, regres=((168, 70), (8760, 20)))
   
    try:
        if regularizer == None:
            model = ElasticNet()
        else:
            model = ElasticNet(l1_ratio=regularizer)
        model.fit(data[features_regions].values, data.taxi_call_num.values)
        r2_score = model.score(data[features_regions].values, data.taxi_call_num.values)
    except LinAlgError as e:
        print(region, e)
    
    return model, r2_score

##### write a prediction function - since features contains information from past - values of taxi calls from previous hours - the prediction process must be sequential - first we predict the value of the calls in the first unknown hour, then in the second (since among the features there will be values that utilize the value of taxi calls predicted in the previous step) and so forth

In [16]:
def predict(region, model, end_t_prediction, features):
    '''feature_constructor(region, regres, dataframe=taxi, weather=test_weather)'''
    data, fourier_shape = feature_constructor(region=region, regres=((168, 70), (8760, 20)), dataframe=taxi_prediction, 
                               weather=pd.concat([test_weather, prediction_weather], axis=0))
    data = data.loc[:end_t_prediction]
    #print("full data shape: ", data.shape)
    '''2016 May 1 00:00:00'''     '''2014-05-01 00:00:00'''
    for h in pd.date_range('2016 May 1 00:00:00', end_t_prediction, freq = 'h'):
        #print(h, end=" ")
        d = [("d_before", 24), ("two_d_before", 48), ("three_d_before", 72), ("four_d_before", 96), ("five_d_before", 120), 
         ("six_d_before", 144), ("one_h_before", 1), ("two_h_before", 2), ("three_h_before", 3), ("four_h_before", 4), 
         ("five_h_before", 5), ("six_h_before", 6), ("seven_h_before", 7), ("eight_h_before", 8), ("nine_h_before", 9), 
         ("ten_h_before", 10), ("eleven_h_before", 11), ("twelve_h_before", 12)]
        
        for name, value in d:
            data.ix[h, name] = data.loc[h - pd.Timedelta(hours=value)]['taxi_call_num']
                   
        dd = [("half_d_cumsum", (1, 12)), ("d_cumsum", (1, 24)), ("week_cumsum", (1, 168)), ("four_w_cumsum", (1, 168*4))]
            
        for name, value in dd:
            data[name] = data["taxi_call_num"].shift(1).rolling(min_periods=value[0], window=value[1]).sum()
                
        
        ''' take only single feature columns '''
        multicross_coll = itertools.combinations(data.columns[fourier_shape : fourier_shape + 26].values, 2)
        for pair in multicross_coll:
            data.ix[h, pair[0] + "_" + pair[1]] = data.ix[h, pair[0]] * data.ix[h, pair[1]] 
        
        p = model.predict((data.loc[h][features].values).reshape(1, -1))
        #p_ = prediction(model, data.loc[h][features])
        
        data.ix[h, 'taxi_call_num'] = p
    
    return data

In [17]:
def plot_bars(x, y1, y2):
    plt.figure(figsize=(len(x) // 2, 10))
    ax = plt.subplot(111)
    a = ax.bar(np.array(x), y1, width=0.4, color='g', align='center')
    b = ax.bar(np.array(x)+0.4, y2, width=0.4, color='b', align='center')
    
    ax.set_title('compare r2 features with and without interaction')
    ax.set_xticks(np.add(x, 0.1)) # set the position of the x ticks
    ax.set_xticklabels(x)
    ax.set_ylim([0.2, 1.1])
    ax.grid()
    ax.legend([a, b], ['without interaction', 'with interaction'] )

    plt.show()

In [18]:
def plot_bars_1(x, y, t, legend):
    num_of_regularizators = len(y[0])
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']
    
    c = colors[:num_of_regularizators]
    plt.figure(figsize=(len(x) // 2, 10))
    ax = plt.subplot(111)
    bars = []
    for i in range(num_of_regularizators):
        bars.append(ax.bar(np.array(x) + i * 0.9 / num_of_regularizators, [z[i] for z in y] , width=0.9 / num_of_regularizators, 
                           color=c[i], align='center'))
       
    ax.set_title(t)
    ax.set_xticks(np.add(x, 0.1)) # set the position of the x ticks
    ax.set_xticklabels(x)
    ax.set_ylim([0.2, 1.1])
    ax.grid()
    ax.legend(bars, legend )
    
    plt.show()

##### function calculates r2 score for train and test data

In [19]:
def get_r2_mse(reg_lst, end_period, f):
    r2_train, r2_test = [], []
    for reg in reg_lst:
        print("REGION: ", reg, end=" ")
        model, r2 = train_linear(region=reg, features_regions=f[reg], regularizer=None)
        #'2016 May 7 23:00:00'
        predicted_df = predict(region=reg, model=model, end_t_prediction='2016-05-07T23:00:00.000000000', features=f[reg])
        r2_train.append(r2)
        r2_test.append(r2_score(verify_taxi.loc[: end_period][region_map[reg]].values, 
                                predicted_df.loc['2016-05-01T00:00:00.000000000': end_period]['taxi_call_num'].values))
        
    return r2_train, r2_test

In [2]:
%%time
''' without combinations '''
r2_train, r2_test = get_r2_mse(list(range(102)), end_period='2016-05-07T23:00:00.000000000', f=features)

In [3]:
%%time
''' with combinations '''
r2_train_combination, r2_test_combination = get_r2_mse(list(range(102)), end_period='2016-05-07T23:00:00.000000000', f=features)

In [122]:
r2_train = [0.8828682542858884, 0.9172727895348061, 0.8958284058058114, 0.8564679315007155, 0.9172702053242211, 0.9238761403489262, 0.9245743639344238, 0.9186438670344894, 0.9165854069023301, 0.8531307287196916, 0.8822928385747301, 0.5445862515012924, 0.737795208449946, 0.7427515820751276, 0.627223933087248, 0.6867185136443997, 0.9355751029155259, 0.9344645333382176, 0.9350922696224985, 0.9397294405941646, 0.8877490032017111, 0.8822440498567612, 0.7260704932931017, 0.7900965402460783, 0.6490682175474907, 0.742647809534505, 0.6419115777695931, 0.6704319815036738, 0.49158460045219166, 0.9200873002400027, 0.9331005538543694, 0.9169442105152261, 0.9293292572528954, 0.9389253681876788, 0.9280922684722738, 0.9348765444770905, 0.9259266968211612, 0.9187080877913216, 0.7190025161772131, 0.6194481142608473, 0.6994092783778292, 0.5882910914547493, 0.7717987462818351, 0.8984147565884056, 0.9416686290933015, 0.9565762509413562, 0.935634905713092, 0.8940446255304948, 0.9165244302596101, 0.9178307107914383, 0.8430284411204677, 0.883698584401781, 0.86293424353258, 0.8648597683479439, 0.9259009582296839, 0.9405388272763889, 0.9525672173340308, 0.9100415284029566, 0.8740668107385644, 0.7947355760379816, 0.7860930811957251, 0.4805074699869226, 0.8468770166334152, 0.813463443199919, 0.7504003843306092, 0.5610800998468819, 0.8093898482453531, 0.9308075298925855, 0.9088101156585764, 0.9140129279595308, 0.7095512617675951, 0.6511441026119196, 0.563291507144552, 0.48836390566715415, 0.5290846447264849, 0.7712567621013248, 0.3754012397612454, 0.936457693855627, 0.7013961969515146, 0.4269605147087403, 0.6385358080131266, 0.6293862871244242, 0.5774133747211591, 0.3868191661979015, 0.345306877730041, 0.5003920433058657, 0.6096360705575806, 0.6805703127376723, 0.7065396151233241, 0.6273809896411797, 0.6513280810054037, 0.6299370495403791, 0.5467397598316435, 0.5419643537391831, 0.5870048200223346, 0.8463411212760262, 0.8309155926275911, 0.8552484395501649, 0.5765684326838676, 0.8263421461231908, 0.7050756238386986, 0.680852178054424]
r2_test = [0.7041670495620198, 0.7826207109085652, 0.7472100891242985, 0.6814644834312122, 0.7527051714320598, 0.8082745673131346, 0.8172599277437189, 0.7965354980078774, 0.7470662362356844, 0.6541801190444863, 0.676661887244955, 0.08551752599324847, 0.3815218540435733, 0.49316368032377866, 0.2555134227604523, 0.5282916091828835, 0.8066803626687811, 0.8276633062194787, 0.8216866072280198, 0.8310425889598777, 0.7277613311412047, 0.7538764233192703, 0.5804371755074169, 0.561686446684475, 0.3593625621027945, 0.47485566060577833, 0.4184056851114325, 0.400851928945949, 0.1457823362954418, 0.7590219065071123, 0.8225253063321649, 0.7671759124105759, 0.8238741863570279, 0.8221045095661763, 0.8270806274175406, 0.8554473821920261, 0.8381962521804704, 0.8425567542610259, 0.4828930324397047, 0.4365844952959137, 0.4668295717031704, 0.3348417689945532, 0.6150507034158859, 0.7575929112030144, 0.8657583763735999, 0.8997903186860989, 0.8428053545719997, 0.8696968195427662, 0.830943545681132, 0.8056901768340577, 0.6910211193319908, 0.7782959253080418, 0.6429149838405037, 0.7382509255123632, 0.7918337267016977, 0.8490281427790631, 0.91322201665524, 0.8271314660889119, 0.8383246971378012, 0.6125971025467913, 0.48394241508211067, 0.24779803177367377, 0.6296928255769245, 0.6453906952202757, 0.4947357666086196, 0.3496609553225124, 0.7114581452277537, 0.8510649639832403, 0.8017424708668623, 0.8373184140085477, 0.4987031770089144, 0.3526597319534217, 0.17305191376995788, 0.18025719013840358, 0.17644644766515583, 0.6292382202246973, -0.0022613915943778817, 0.46629366696557395, 0.43997970997845326, 0.25421094608318096, 0.3022398014330693, 0.10352170068018296, 0.383356952972145, 0.13435251545283933, -0.01565762602382703, 0.27600547619667415, 0.35386631810781355, 0.5467124469885376, 0.5662331347506229, 0.46329776408739154, 0.49484754663362485, 0.5205202270982252, 0.48034749166607027, 0.3917848468636197, 0.31723854297445697, 0.8304121213344372, 0.713557384010598, 0.8110702769316822, 0.4137743413378525, 0.7789624061573771, 0.7358265976560774, 0.5748925081060966]

r2_train_combination = [0.8928206234583039, 0.9366225299618696, 0.9270857881016785, 0.8807871256837689, 0.9295808884558581, 0.9441170615522094, 0.9443248543476761, 0.9402130892718087, 0.9463138617725376, 0.8837689923701616, 0.9095086412998792, 0.5976007649864097, 0.7912185795686566, 0.7926482285476179, 0.6857197149713452, 0.7313272516874978, 0.9628857282742238, 0.9521471145514778, 0.9502000847333769, 0.9530002311760647, 0.9211048544081404, 0.9070783527324904, 0.7488903622951304, 0.8320477728712637, 0.6927584131855091, 0.7845199826882561, 0.6779481015577087, 0.7132586009526883, 0.5281549242528027, 0.9565384571751123, 0.9546957567273602, 0.9377467629044605, 0.951019294849568, 0.9534460351230999, 0.9424776301537328, 0.948121185467951, 0.9404282773034159, 0.9335529171247754, 0.7502352930261112, 0.657219929003697, 0.7424145040375143, 0.6407445043715788, 0.8087188187178288, 0.9210936881509261, 0.9578273346529524, 0.9670684281863504, 0.9487845637827191, 0.9067311855537313, 0.9334712572156629, 0.934388518927142, 0.883432122229782, 0.9156505527828334, 0.899382518425569, 0.9031420411266255, 0.9494123038265169, 0.9644910598617416, 0.9681105636503982, 0.9386468135188974, 0.902707795427867, 0.8372300035610692, 0.8126783625473829, 0.5256346411081719, 0.8868075750628222, 0.8621230694992503, 0.7976746090166785, 0.5828131928116351, 0.8457781372085257, 0.9597582055550201, 0.9357531950904874, 0.9446909900328677, 0.7413127985119208, 0.7151059622514507, 0.609817164493875, 0.5315423342898812, 0.5943198360796367, 0.8156696434905241, 0.40040106040440504, 0.9607837097910891, 0.7531263428764623, 0.4769441549081296, 0.6955742794145994, 0.6921411383768044, 0.6423541713258696, 0.4458524738059312, 0.37230512269518246, 0.5445477312029641, 0.6529632818176422, 0.728696460503182, 0.7583813393627021, 0.6858143046303429, 0.7088772848092091, 0.6737768178064811, 0.5764010559192799, 0.6105227189377385, 0.6448817750411857, 0.875038114862852, 0.8708267334107598, 0.8637633627542449, 0.5937488326822911, 0.8442566923413284, 0.7241515187593897, 0.7073816377294444]
r2_test_combination = [0.7614445068277522, 0.8456230964548673, 0.8373273970502155, 0.6842407226838989, 0.7906913341813564, 0.8135395560452116, 0.8406888725245177, 0.878545915913743, 0.807995241471031, 0.6992505020610814, 0.11733779091768304, 0.10442847141436074, 0.422875750508247, 0.5619643173369449, 0.36380930639862497, 0.5977418956743251, 0.8537123047000162, 0.8619061432928969, 0.7762331652646061, 0.7620633509176724, 0.8346972516180788, 0.7957521210042183, 0.5553177142486119, 0.7281530600679164, 0.4177320654424669, 0.5741893327412693, 0.44309564718058037, 0.38755830608792396, 0.11009523225651674, 0.3465024091867521, 0.8111904311730687, 0.813482091436146, 0.8712199201019326, 0.8523887578374749, 0.8809508938097806, 0.8927624560499309, 0.8709077553228373, 0.8712842949984143, 0.44095608300167044, 0.4636515998987699, 0.5416426700806223, 0.5100799641180829, 0.7121409353648299, 0.8573874086325193, 0.8865727537080349, 0.8997576677839236, 0.8970324734711981, 0.8837304994929076, 0.8690070534923942, 0.8258534618796842, 0.8051352894426311, 0.8359902570491584, 0.09467761770744398, 0.7948843974360882, 0.8107896382032852, 0.9241780072249333, 0.9600594537600539, 0.8754583643267457, 0.8942583717052577, 0.7095292863684621, 0.3762271121419206, 0.30112938352658924, 0.7553463837721934, 0.7641795493271875, 0.4936334647916666, 0.38121140218853866, 0.7845382990108973, 0.906007939544268, 0.8789891014048911, 0.8998537960998633, 0.4944887037666478, 0.4683707933002982, 0.17570201448373401, 0.07141377002310234, 0.08927086605141099, 0.7696503390002076, -0.041326022559851294, 0.28852229190641476, 0.5532360228687525, 0.3302460013998064, 0.24762580098817577, 0.0872333236987608, 0.254899810083545, 0.1195400849166115, -0.1077477829353588, 0.30342240386133645, 0.41674464509995335, 0.46456139741390245, 0.5668680322315064, 0.5673525893208107, 0.601364520870562, 0.5299148928058139, 0.5387329943060561, 0.37041075360155307, 0.22252826565181616, 0.8557323991271579, 0.7956860156818948, 0.8144989925163014, 0.40428332493116115, 0.8037395135286061, 0.7223969253158463, 0.5590806934107064]

In [125]:
plot_bars(range(102), r2_train, r2_train_combination)
#double click on image makes it bigger 

In [126]:
plot_bars(range(102), r2_test, r2_test_combination)
#double click on image makes it bigger 

##### СЃР°se with binary combinations show slightly better result on all regions at train data and better results on mostly region on test set (except region # 10, 28, 29, 38, 52, 73, 74, 77, 80, 82, 94.  The number of features is greatly increased

##### in the future we will use features interaction

##### use  XGBoostRegressor insted of ElasticNet

In [20]:
def train_xgboost(region, features_regions, parameters=None):
    data, _ = feature_constructor(region=region, regres=((168, 70), (8760, 20)))
   
    if parameters == None:
        model = XGBRegressor()
    else:
        model = XGBRegressor(**parameters)
        xgb_param = model.get_xgb_params()
        xgtrain = xgb.DMatrix(data[features_regions].values, data.taxi_call_num.values)
        cvresult = xgb.cv(params=xgb_param, dtrain=xgtrain, num_boost_round=model.get_params()['n_estimators'], nfold=5,
            metrics='rmse', early_stopping_rounds=50)
        model.set_params(n_estimators=cvresult.shape[0])
    
    model.fit(data[features_regions].values, data.taxi_call_num.values)
    r2_score = model.score(data[features_regions].values, data.taxi_call_num.values)
       
    return model, r2_score    

In [21]:
def get_r2_xgboost(reg_lst, end_period, f, plot=False, parameters=None):
    r2_train, r2_test = [], []
    for reg in reg_lst:
        print("REGION: ", reg, end=" ")
        if parameters:
            model, r2 = train_xgboost(region=reg, features_regions=f[reg], parameters=parameters)
        else:
            model, r2 = train_xgboost(region=reg, features_regions=f[reg], parameters=None)
        """ '2016 May 7 23:00:00' """
        predicted_df = predict(region=reg, model=model, end_t_prediction='2016-05-07T23:00:00.000000000', features=f[reg])
        r2_train.append(r2)
        r2_test.append(r2_score(verify_taxi.loc[: end_period][region_map[reg]].values, 
                                predicted_df.loc['2016-05-01T00:00:00.000000000': end_period]['taxi_call_num'].values))
        
        if plot:
            names = f[reg]
            
            pyplot.figure(figsize=((len(f[reg]) - 1)//4, 7))
            significance_list = sorted(zip(names, model.feature_importances_), key=lambda x: x[1], reverse=True)
            pyplot.bar(range(len(model.feature_importances_)), [x[1] for x in significance_list])
            pyplot.title('Feature Importances')
            pyplot.xticks(np.arange(len(names)), [x[0] for x in significance_list], rotation=90)
            plt.ylabel('Feature Importance Score')
            pyplot.show()
        
    return r2_train, r2_test

##### look at r2 score values for train and test data for XGBoost Regressor with default parameters in case of features with and without binary combinations

In [133]:
xgb_r2_train, xgb_r2_test = get_r2_xgboost(reg_lst=list(range(102)), end_period='2016-05-07T23:00:00.000000000', f=features)

In [111]:
xgb_r2_train = [0.9187738653338163, 0.9529657261914531, 0.9486581840398112, 0.9124231385728379, 0.9494596806074327, 0.9603551715734012, 0.9621501291816412, 0.9617570776955187, 0.9656743216315316, 0.9220016239281933, 0.9377491837037846, 0.7306882169427737, 0.8640346425835592, 0.8545192605197648, 0.7735519543392956, 0.8060930376422659, 0.9768537483855108, 0.9702977540642541, 0.9652455230666271, 0.9667075689221555, 0.9446573129411604, 0.9416759610767318, 0.8253850714439432, 0.87187376108258, 0.8080702382928476, 0.8659666040641109, 0.7596311226606985, 0.7788873614144719, 0.6742332554087502, 0.9769432967640911, 0.9726637971955063, 0.9593149916731497, 0.9658009446708915, 0.9648844347975996, 0.9601174985601986, 0.9638650133600041, 0.9582767743216727, 0.9513267114203592, 0.8353820100742462, 0.7743621738386469, 0.8189887856309954, 0.7474057073225154, 0.8606902638009406, 0.9454736379038716, 0.9680823564057295, 0.9763103326003071, 0.9608597943926355, 0.9317570348126842, 0.9532497329819222, 0.9508500780259564, 0.9174099461168184, 0.9458259446469038, 0.9355192748914768, 0.9285414921861173, 0.9675306582942681, 0.9760930639684546, 0.9769880686190456, 0.9554948798047573, 0.930706869013777, 0.883787235093459, 0.8611641920629078, 0.6155981799657699, 0.9333017598147432, 0.9165373824047262, 0.870079565480486, 0.6765597238539769, 0.8871937432305343, 0.9731248732140917, 0.9544592732929246, 0.9621844719236714, 0.8089752700572186, 0.8000887796390506, 0.7123627948354205, 0.6471259174061295, 0.7133705234905114, 0.8901388239693518, 0.5422581782128657, 0.9764832748048451, 0.8165215548551832, 0.6030484137659551, 0.7651960582654145, 0.7903875035925538, 0.7444652922891182, 0.6109566981928141, 0.5266899832115067, 0.6711517284700926, 0.7680015183620916, 0.8003078371671102, 0.8447477887719921, 0.7976453065612034, 0.8078442807801355, 0.7667213345244693, 0.7060050331976192, 0.7007641229848158, 0.7315065261935636, 0.8985122146161525, 0.8995690871614344, 0.8848680462274886, 0.6630274971469332, 0.8707997200640665, 0.7673380457549045, 0.7523188243641852]
xgb_r2_test = [0.846267990116081, 0.9182833977631235, 0.9137773780752378, 0.8074639335888979, 0.84835878555008, 0.9053683059707458, 0.9534985166758762, 0.9549508749691644, 0.921373429778927, 0.8638048815133503, 0.6569352206398658, 0.4083828911909553, 0.7679969495823894, 0.8275021551739742, 0.6347871581221186, 0.7844155169232148, 0.95654851945247, 0.9354221334979234, 0.9454195149061915, 0.9460854542083843, 0.8717953112322472, 0.8709368576758834, 0.6434254275536095, 0.7552842942390228, 0.005699478868773089, 0.6581457868562373, 0.6959826752560629, 0.6747371678113483, 0.43663600510804845, 0.9312973116083084, 0.9608383448988159, 0.9153082853673202, 0.9264081811864497, 0.9143720526875345, 0.9478758788199498, 0.9455279279614062, 0.947774745013847, 0.9395208672878422, 0.6578461087975969, 0.6272951257822438, 0.6279742084592188, 0.6562084206772616, 0.8400019262026777, 0.8910508617335077, 0.9437529705571331, 0.9708739370713586, 0.9351016702088932, 0.9326758062597789, 0.9425686046588413, 0.9281175058350962, 0.8848014256384793, 0.9093727896526531, 0.6793690665949516, 0.913828887877294, 0.947233710192897, 0.9264064694220446, 0.9035447831212351, 0.9056009580697523, 0.943181054747099, 0.8501371524759224, 0.7270856302973967, 0.5329674598324502, 0.7687276183362294, 0.5704133221532233, -0.316360713259189, 0.5136253291484263, 0.8539408908149256, 0.9569384417810032, 0.9186168861031792, 0.914898053614764, 0.6680975836953464, 0.6484687863183856, 0.5340941507363329, 0.40358796555653786, 0.5370238683329371, 0.34354051343247094, 0.15516560449546357, 0.57137279657963, 0.6335512167472224, 0.5123856832571236, 0.6445644908806449, 0.5615696802429018, 0.736779270171595, 0.45255905551380604, 0.28810422037302186, 0.6629206448647468, 0.5083468612323649, 0.7754382054476102, 0.8292270639207846, 0.6923946084417014, 0.2977782206463232, 0.707837129381887, 0.700087632051424, 0.5867483882264269, 0.4368899580993685, 0.8959280596220356, 0.8893635131621992, 0.8495817816990476, 0.502953432759789, 0.851816227321684, 0.7445121623832409, 0.6890876890840769]

In [134]:
plot_bars(range(102), r2_train_combination, xgb_r2_train)
#double click on image makes it bigger

In [135]:
plot_bars(range(102), r2_test_combination, xgb_r2_test)
#double click on image makes it bigger

### Tune xgboost parameters

##### in this notebook two setting parameters approaches will be used: first of them - sequential selection of parameters. use feature set with combinations

#### First version of tuning xgboost parameters

default parameters are:

xgboost.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='reg:linear', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)

#### Fix learning rate and number of estimators

max_depth = 5 : This should be between 3-10 <br>
min_child_weight = 1 : default value <br>
gamma = 0 : A smaller value like 0.1-0.2 can also be chosen for starting. This will anyways be tuned later <br>
subsample, colsample_bytree = 0.8 : This is a commonly used used start value. Typical values range between 0.5-0.9 <br>
scale_pos_weight = 1: default value <br>

##### train on may 2014 - april 2016 validate on 1-st week of may 2016, choose three regions - 8, 28, 76, region 8 showed good result on both train and test sets, remained two regions have average results on train data and bad results on test dataset
##### plot shows histogram of features sorted by their "prediction power"

In [194]:
default_train, default_test = np.array(xgb_r2_train)[[8, 28, 76]], np.array(xgb_r2_test)[[8, 28, 76]]

#### 1) tune number of estimators

In [190]:
xgb_r2_train1, xgb_r2_test1 = get_r2_xgboost(reg_lst=[8, 28, 76], end_period='2016-05-07T23:00:00.000000000', f=features, plot=True, 
                                             parameters={'learning_rate':0.1, 'n_estimators':400, 'max_depth':5, 
                                                         'min_child_weight':1, 'gamma':0, 'subsample':0.8, 'colsample_bytree':0.8,
                                                         'objective':'reg:linear', 'scale_pos_weight':1, 'seed':27})

#double click on image makes it bigger

In [195]:
train_lr_01_n_est_400, test_lr_01_n_est_400 = xgb_r2_train1, xgb_r2_test1
print(default_train, default_test)
print(train_lr_01_n_est_400, test_lr_01_n_est_400)

In [ ]:
xgb_r2_train2, xgb_r2_test2 = get_r2_xgboost(reg_lst=[8, 28, 76], end_period='2016-05-07T23:00:00.000000000', f=features, plot=False, 
                                             parameters={'learning_rate':0.1, 'n_estimators':600, 'max_depth':5, 
                                                         'min_child_weight':1, 'gamma':0, 'subsample':0.8, 'colsample_bytree':0.8,
                                                         'objective':'reg:linear', 'scale_pos_weight':1, 'seed':27})

#double click on image makes it bigger

In [193]:
train_lr_01_n_est_600, test_lr_01_n_est_600 = xgb_r2_train2, xgb_r2_test2
print(train_lr_01_n_est_600, test_lr_01_n_est_600)

##### 1) learn rate 0.1 number of estimators: default <br>train data r2 [0.96567432 0.67423326 0.54225818] test r2 data [0.92137343 0.43663601 0.1551656 ] <br>2) learn rate 0.1 number of estimators: 400 <br> train data r2 [0.99127, 0.78505, 0.63436] test r2 data [0.93305, 0.51544, 0.239238] <br>3) learn rate 0.1 number of estimators: 600  <br> train data r2 [0.99316, 0.78505, 0.63436] test r2 data [0.92929, 0.51544, 0.23923] <br>

##### increasСѓ the number of estimators to 400 make the r2 score higher for train and test set. further increase does not lead to any improvement. 
#### best variant:  lr=0.1, n_estimators=400

#### 2) Tune max_depth and min_child_weight

In [254]:
#param_1 = {'max_depth':list(range(3,10,2)), 'min_child_weight':list(range(1,6,2))}
train_lst, test_lst = {}, {}

In [1]:
for i in list(range(3,10,2)):
    for j in list(range(1,6,2)):
        xgb_r2_train, xgb_r2_test =get_r2_xgboost(reg_lst=[8, 28, 76], end_period='2016-05-07T23:00:00.000000000', f=features, 
                                                  plot=False, parameters={'learning_rate':0.1, 'n_estimators':400, 'max_depth':i, 
                                                                         'min_child_weight':j, 'gamma':0, 'subsample':0.8, 
                                                                         'colsample_bytree':0.8, 'objective':'reg:linear', 
                                                                         'scale_pos_weight':1, 'seed':27})
        train_lst[(i, j)], test_lst[(i, j)] = xgb_r2_train, xgb_r2_test

In [306]:
print(train_lst)

In [307]:
print(test_lst)

##### best (max_depth and min_child_weight): (7, 5) <br> train:[0.994854, 0.782799, 0.695982] test: [0.933216, 0.502331, 0.294959]

#### 3)Tune gamma

In [308]:
train_lst1, test_lst1 = {}, {}

In [310]:
for i in [i/10.0 for i in range(0,5)]:
    
    xgb_r2_train, xgb_r2_test =get_r2_xgboost(reg_lst=[8, 28, 76], end_period='2016-05-07T23:00:00.000000000', f=features, 
                                              plot=False, parameters={'learning_rate':0.1, 'n_estimators':400, 'max_depth':7, 
                                                                     'min_child_weight':5, 'gamma':i, 'subsample':0.8, 
                                                                     'colsample_bytree':0.8, 'objective':'reg:linear', 
                                                                     'scale_pos_weight':1, 'seed':27})
    train_lst1[i], test_lst1[i] = xgb_r2_train, xgb_r2_test

In [311]:
print(train_lst1)

In [312]:
print(test_lst1)

##### best gamma 0.1  <br> train: [0.9948547140718738, 0.782799018968261, 0.6903512552540485]   test:   [0.9332160058506537, 0.502331351170488, 0.3002932604107048]

#### 4) Tune subsample and colsample_bytree

In [313]:
train_lst2, test_lst2 = {}, {}

In [2]:
''' param_3 = {'subsample':[i/10.0 for i in range(6,10)], 'colsample_bytree':[i/10.0 for i in range(6,10)]} '''
for i in [i/10.0 for i in range(6,10)]:
    for j in [i/10.0 for i in range(6,10)]:
        xgb_r2_train, xgb_r2_test =get_r2_xgboost(reg_lst=[8, 28, 76], end_period='2016-05-07T23:00:00.000000000', f=features, 
                                                  plot=False, parameters={'learning_rate':0.1, 'n_estimators':400, 'max_depth':7, 
                                                                         'min_child_weight':5, 'gamma':0.1, 'subsample':i, 
                                                                         'colsample_bytree':j, 'objective':'reg:linear', 
                                                                         'scale_pos_weight':1, 'seed':27})
        train_lst2[(i, j)], test_lst2[(i, j)] = xgb_r2_train, xgb_r2_test

In [315]:
print(train_lst2)

In [316]:
print(test_lst2)

##### best (subsample and colsample_bytree) - (0.9, 0.6) <br>  train: [0.9950744172838145, 0.8028502415229332, 0.7043995379367959]   test:  [0.9324777016916048, 0.5046125249821583, 0.3015860210902518]

##### 5) Tuning Regularization Parameters

In [317]:
train_lst3, test_lst3 = {}, {}

In [319]:
''' param_4 = {'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100, 200]} '''

for i in [1e-5, 1e-2, 0.1, 1, 100, 200]:
    
    xgb_r2_train, xgb_r2_test =get_r2_xgboost(reg_lst=[8, 28, 76], end_period='2016-05-07T23:00:00.000000000', f=features, 
                                              plot=False, parameters={'learning_rate':0.1, 'n_estimators':400, 'max_depth':7, 
                                                                     'min_child_weight':5, 'gamma':0.1, 'subsample':0.9, 
                                                                     'colsample_bytree':0.6, 'objective':'reg:linear', 
                                                                     'scale_pos_weight':1, 'seed':27, 'reg_alpha':i})
    train_lst3[i], test_lst3[i] = xgb_r2_train, xgb_r2_test

In [320]:
print(train_lst3)

In [321]:
print(test_lst3)

##### significant regularization makes r2 score lower 

##### best result: train: [0.995074, 0.802850, 0.704399]  test:  [0.932477, 0.5046123, 0.301586]

##### 6) try to vary 'learning_rate' and 'n_estimators'

In [322]:
xgb_r2_train, xgb_r2_test =get_r2_xgboost(reg_lst=[8, 28, 76], end_period='2016-05-07T23:00:00.000000000', f=features, 
                                              plot=False, parameters={'learning_rate':0.1, 'n_estimators':600, 'max_depth':7, 
                                                                     'min_child_weight':5, 'gamma':0.1, 'subsample':0.9, 
                                                                     'colsample_bytree':0.6, 'objective':'reg:linear', 
                                                                     'scale_pos_weight':1, 'seed':27, 'reg_alpha':0})

In [324]:
print(xgb_r2_train, xgb_r2_test)

In [325]:
xgb_r2_train, xgb_r2_test =get_r2_xgboost(reg_lst=[8, 28, 76], end_period='2016-05-07T23:00:00.000000000', f=features, 
                                              plot=False, parameters={'learning_rate':0.05, 'n_estimators':800, 'max_depth':7, 
                                                                     'min_child_weight':5, 'gamma':0.1, 'subsample':0.9, 
                                                                     'colsample_bytree':0.6, 'objective':'reg:linear', 
                                                                     'scale_pos_weight':1, 'seed':27, 'reg_alpha':0})

In [326]:
print(xgb_r2_train, xgb_r2_test)

### Best result for region 8, 28, 76 on train and test datasets:
##### 'learning_rate':0.1, 'n_estimators':400, 'max_depth':7, 'min_child_weight':5, 'gamma':0.1, 'subsample':0.9, 'colsample_bytree':0.6, 'objective':'reg:linear', 'scale_pos_weight':1, 'seed':27, 'reg_alpha':0}

##### try to apply this results to all regions

In [31]:
xgb_r2_train_select, xgb_r2_test_select = get_r2_xgboost(reg_lst=list(range(102)), end_period='2016-05-07T23:00:00.000000000', 
                                                         f=features, plot=False, 
                                                         parameters={'learning_rate':0.1, 'n_estimators':400, 'max_depth':7, 
                                                                     'min_child_weight':5, 'gamma':0.1, 'subsample':0.9, 
                                                                     'colsample_bytree':0.6, 'objective':'reg:linear', 
                                                                     'scale_pos_weight':1, 'seed':27, 'reg_alpha':0})

##### apply selected parameters to all of 102 regions and compare results for default and selected parameters on train and test dataset 

In [119]:
xgb_r2_train_select = [0.9615885027124841, 0.9857616355969546, 0.9839869622603288, 0.9547946331919521, 0.9776373279354994, 0.9907702658205959, 0.9906443690735878, 0.9932155369084056, 0.9950744172838145, 0.9735566466160163, 0.9975920238141726, 0.8888566228811084, 0.9572395521467886, 0.9380429509558402, 0.8897980199257776, 0.8970384201254856, 0.9961779219064939, 0.9945980780322431, 0.9919702173547124, 0.9944442693905416, 0.9880652899220782, 0.9899131993528317, 0.9701137627617081, 0.9397822541406765, 0.9173299932780845, 0.9362277489627314, 0.8895370267250328, 0.9001301029083767, 0.8028502415229332, 0.9956988838036093, 0.9965738903985151, 0.9926006436643713, 0.9928788637247757, 0.9975233919170534, 0.9973339449957281, 0.9950053587016471, 0.9915067841806938, 0.9841282055278555, 0.9229074000604003, 0.9019691358210311, 0.9128693472277805, 0.8548198030387482, 0.950011619464288, 0.9842199950486504, 0.9976011787849229, 0.9973006934923091, 0.9927256222575493, 0.9704788981099759, 0.9868190716663067, 0.9794477263867544, 0.9600638771503444, 0.975785605891054, 0.9799391067964461, 0.9790776175501319, 0.9908043197875398, 0.997333678260201, 0.9952707659015052, 0.9902276226072471, 0.9669366077879812, 0.9489053532445799, 0.9436950035590704, 0.7617280270977793, 0.9799855924720253, 0.9686811585711154, 0.935384846488926, 0.8025154675701816, 0.9352742619561208, 0.9938750444316548, 0.9955868627939475, 0.9934326189592231, 0.9184251674809054, 0.9075071589463114, 0.8491531218294001, 0.8235636491184508, 0.8571194914093826, 0.9501893102017939, 0.7043995379367959, 0.9937994723151049, 0.8965704607409007, 0.7515494014798442, 0.8835852584742592, 0.9280473617034907, 0.8842457353747246, 0.7623367867261157, 0.7079652876754895, 0.8008857510581994, 0.8727082414879641, 0.9018209877127256, 0.9139158016856626, 0.9023397022625492, 0.9178062599867707, 0.8862313381934718, 0.8286144147143286, 0.8444378252349188, 0.8195612023631897, 0.9648756757865069, 0.9587813494723298, 0.9394641596675674, 0.7825566088875604, 0.9381929270078211, 0.8545594247502719, 0.8927910044667434]
xgb_r2_test_select = [0.8460616038913957, 0.919468528574407, 0.928132850347398, 0.8147714482673521, 0.8779452370644896, 0.9395636250208023, 0.9569134349230388, 0.960305059398311, 0.9324777016916048, 0.9022636457520733, -0.30989438849981976, 0.5497328077973995, 0.7736071188728529, 0.8217436084581811, 0.6451739156978511, 0.8125017969737418, 0.9595306135537541, 0.9537797870945905, 0.9629210258552509, 0.9583935241013344, 0.8868811076867513, 0.8783423853918625, 0.6562439269055169, 0.7641692706179836, -0.24884277753392348, 0.5318885154781101, 0.7337920672061702, 0.6902046998494746, 0.5046125249821583, 0.9434835729885953, 0.9384813761834019, 0.9321126963902824, 0.9527105752251285, 0.950066553997302, 0.9469466317155456, 0.952290995823351, 0.9549780004651148, 0.9549701112163929, 0.6771992731769962, 0.5922869559842322, 0.5307018733587638, 0.6752635582179479, 0.8514206243344502, 0.9077355864940169, 0.9583918789009391, 0.979638640925733, 0.9553541396960655, 0.9373667963482191, 0.9547493723240124, 0.9579123044509439, 0.901421834340595, 0.8305143604550911, 0.20982684372372895, 0.9357021143396751, 0.9668181258690323, 0.9638755950279217, 0.974912312385217, 0.9419656662690177, 0.9568968908983774, 0.8828498049526178, 0.7193372904304971, 0.5422093432244983, 0.5635039248733373, 0.7328675635417772, -3.611915354570633, 0.4992478179021078, 0.8659092773986117, 0.9627024081594028, 0.9512305954245451, 0.970044840880199, 0.7248175939357957, 0.7327568326825498, 0.6007064658784823, 0.5356110757045528, 0.619500953362346, -0.5701307454142928, 0.3015860210902518, 0.5664455687975665, 0.6638602025567494, 0.5132065196439555, 0.5972628673816721, 0.7069008979689755, 0.7449486800155234, 0.5639382822947964, 0.3516955504114244, 0.5974621768584285, -2.1967245242888245, 0.7596129278540436, 0.8175084719608012, 0.6379725769022647, 0.40621094127653545, 0.7117716038242998, 0.7064763773434151, 0.6147730906826497, 0.44420359795276, 0.8908676297970316, 0.88605813936086, 0.8423231721783956, 0.48763903399183883, 0.8460485341672875, 0.7335094785853361, 0.6845745363532589]

In [38]:
plot_bars(range(102),xgb_r2_train, xgb_r2_train_select)

In [157]:
plot_bars(range(102), xgb_r2_test, xgb_r2_test_select)

##### result is better for most of regions

### Next step - try to use Arima on residuals fron linear regression. Take Arima parameters and fourier components from previous week results.

##### We can use ElasticNet or XGBoostRegressor as a final regressor model

In [22]:
from scipy.stats import boxcox
import statsmodels.api as sm

In [23]:
def invboxcox(y,lmbda):
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda*y+1)/lmbda))

In [40]:
# regions belong to each cluster
clusters = {0: [9, 37, 48, 59, 60, 68, 70, 86], 
            1: [7, 11, 12, 13, 14, 15, 21, 27, 28, 31, 41, 61, 71, 72, 73, 84], 
            2: [74, 76, 77, 79, 80, 81, 82, 83, 85, 87, 88, 91], 
            3: [0, 3, 4, 5, 6, 17, 18, 19, 32, 33, 34, 35, 36, 44, 45, 95, 96, 97, 98, 99, 100, 101], 
            4: [8, 16, 24, 25, 26, 29, 30, 38, 39, 40, 51, 52, 62, 63, 64, 65, 75, 89, 90, 92], 
            5: [1, 2, 10, 20, 22, 23, 42, 43, 46, 47, 49, 50, 53, 54, 55, 56, 57, 58, 66, 67, 69, 78, 93, 94]}

In [41]:
# [(fourier_period, components_num), (arima)]
parameters = {0: [((168, 80), (8760, 15), (2, 23)), (2, 1, 1, 1, 1, 0)], 
              1: [((168, 70), (8760, 15), (1, 12)), (5, 1, 1, 1, 1, 0)], 
              2: [((168, 70), (8760, 20), (1, 12)), (14, 1, 1, 1, 1, 0)], 
              3: [((168, 60), (8760, 15), (11, 12)), (3, 1, 1, 1, 1, 0)], 
              4: [((168, 60), (8760, 20), (1, 30)), (9, 2, 1, 1, 1, 0)], 
              5: [((168, 60), (1, 30)), (10, 1, 1, 1, 1, 0)]}

In [24]:
def new_feature_constructor(region, params, dataframe=taxi, weather=test_weather):
    ''' params : ((period, component_num), ...) '''
    data = pd.DataFrame(dataframe.iloc[:, region].values, index = dataframe.index, columns=["taxi_call_num"])
    time = np.arange(0, dataframe.shape[0])
    
    ''' fourier components - as was shown in previous week - it was enought approx. 15 year and 70 week components '''
    for component in params[0]:
        for w in range(1, component[1]+1):
            data["sin_w_%d_%d" % (component[0], w)] = np.sin(2*np.pi*w*time/component[0]) 
            data["cos_w_%d_%d" % (component[0], w)] = np.cos(2*np.pi*w*time/component[0])
    data = data.round(3)
    
    #s = 'taxi_call_num ~ ' + ' + '.join([' + '.join(list(data.columns)[1: ])])
    box_cox_params = None
    
    def arima(resids, params):
        try:
            resid_model=sm.tsa.statespace.SARIMAX(resids, order=(params[1][0], params[1][4], params[1][1]), 
                                                  seasonal_order=(params[1][2], params[1][5], params[1][3], 24)).fit(disp=-1)
            resid_prediction = resid_model.fittedvalues
        except ValueError as e:
            print("nonstationarity")
            try:
                min_value = abs(min(resids))
                log_resid, lmbda = boxcox(min_value + 0.1 + resids)
                resid_model=sm.tsa.statespace.SARIMAX(log_resid, order=(params[1][0], params[1][4], params[1][1]), 
                                                      seasonal_order=(params[1][2], params[1][5], params[1][3], 24)).fit(disp=-1)
                resid_prediction = resid_model.fittedvalues
                resid_prediction[resid_prediction == 0] = 0.01
                resid_prediction = invboxcox(resid_prediction, lmbda)
                resid_prediction = (resid_prediction - min_value - 0.1)
                box_cox_params = (lmbda, min_value)
            except ValueError as e:
                print(region, e)
                print("unfortunally boxcox does not help") 
                resid_model = None
                resid_prediction = [0]*time.shape[0]
        return resid_model, resid_prediction        
    
    
    try:
        #m = smf.ols(s, data=data.iloc[:time.shape[0], :])
        #fourier_model = m.fit(cov_type='HC1')
        m = ElasticNet()
        fourier_model = m.fit(data.iloc[:time.shape[0], 1:].values, data.taxi_call_num.values)
        resids = data.taxi_call_num.values - fourier_model.predict(data.iloc[:time.shape[0], 1:].values)
        resid_model, resid_prediction = arima(resids, params)
    except LinAlgError as e:
        print(region, e)  
        #m = smf.ols(s, data=data.iloc[:time.shape[0], :])
        #fourier_model = m.fit_regularized(method='elastic_net', L1_wt=0.5)
        m = ElasticNet(l1_ratio=0.3)
        fourier_model = m.fit(data.iloc[:time.shape[0], 1:].values, data.taxi_call_num.values)
        resids = data.taxi_call_num.values - fourier_model.predict(data.iloc[:time.shape[0], 1:].values)
        resid_model, resid_prediction = arima(resids, params)
        
    
    final_data = pd.DataFrame(dataframe.iloc[:, region].values, index = dataframe.index, columns=["taxi_call_num"])
    #final_data["fourier"] = fourier_model.fittedvalues
    final_data["fourier"] = fourier_model.predict(data.iloc[:time.shape[0], 1:].values)
    final_data["arima_residuals"] = resid_prediction
    
    
    d = [("d_before", 24), ("two_d_before", 48), ("three_d_before", 72), ("four_d_before", 96), ("five_d_before", 120), 
         ("six_d_before", 144), ("one_h_before", 1), ("two_h_before", 2), ("three_h_before", 3), ("four_h_before", 4), 
         ("five_h_before", 5), ("six_h_before", 6), ("seven_h_before", 7), ("eight_h_before", 8), ("nine_h_before", 9), 
         ("ten_h_before", 10), ("eleven_h_before", 11), ("twelve_h_before", 12)]
    
    for name, value in d:
        shift(final_data, shift_period=value, column_name=name)
    dd = [("half_d_cumsum", (1, 12)), ("d_cumsum", (1, 24)), ("week_cumsum", (1, 168)), ("four_w_cumsum", (1, 168*4))]
    
    for name, value in dd:
        cumsum(final_data, min_period=value[0], window=value[1], column_name=name)
    final_data["hour"] = final_data.index.hour
    final_data["week_day"] = final_data.index.dayofweek
    final_data["day"] = final_data.index.day
    final_data["month"] = final_data.index.month
    
    #exclude 1-st column - taxi_call_num
    binary_combinations(final_data, fourier_shape=1)
    final_data["weekend"] = [1 if (final_data.weekday == 5 or final_data.weekday == 6) else 0 for final_data in final_data.index]
        
    final_data = pd.concat([final_data, weather.iloc[:, [0, 1, 3]]], axis=1)
    final_data = final_data.fillna(0.1)
            
    return final_data, fourier_model, resid_model, box_cox_params 

In [25]:
def new_train_linear(region, params, features_regions, regularizer=None, dataframe=taxi, weather=test_weather):
    train_df, fourier_model, resid_model, box_cox_params = new_feature_constructor(region=region, params=params)
   
    try:
        if regularizer == None:
            linear_model = ElasticNet()
        else:
            linear_model = ElasticNet(l1_ratio=regularizer)
        linear_model.fit(train_df[features_regions].values, train_df.taxi_call_num.values)
        r2_score = linear_model.score(train_df[features_regions].values, train_df.taxi_call_num.values)
    except LinAlgError as e:
        print(region, e)
    
    return train_df, linear_model, fourier_model, resid_model, r2_score, box_cox_params 

In [26]:
def new_train_xgboost(region, linear_params, features_regions, xgboost_params=None, dataframe=taxi, weather=test_weather):
    train_df, fourier_model, resid_model, box_cox_params = new_feature_constructor(region=region, params=linear_params)
   
    if xgboost_params == None:
        xgb_model = XGBRegressor()
    else:
        xgb_model = XGBRegressor(**xgboost_params)
        xgb_param = xgb_model.get_xgb_params()
        xgtrain = xgb.DMatrix(train_df[features_regions].values, train_df.taxi_call_num.values)
        cvresult = xgb.cv(params=xgb_param, dtrain=xgtrain, num_boost_round=xgb_model.get_params()['n_estimators'], nfold=5,
            metrics='rmse', early_stopping_rounds=50)
        xgb_model.set_params(n_estimators=cvresult.shape[0])
    
    xgb_model.fit(train_df[features_regions].values, train_df.taxi_call_num.values)
    r2_score = xgb_model.score(train_df[features_regions].values, train_df.taxi_call_num.values)
       
    return train_df, xgb_model, fourier_model, resid_model, r2_score, box_cox_params 

In [27]:
def new_get_r2(reg_lst, clst_num, end_period, f, regressor, linear_l1=None, xgboost_params=None):
    r2_train, r2_test = {}, {}
    for reg in reg_lst:
        print("REGION: ", reg, end=" ")
        if regressor == "linear":
            if linear_l1 == None:
                train_df, model, fourier_model, resid_model, r2, box_cox_params = \
                                               new_train_linear(reg, parameters[clst_num], f[reg])
            else:
                train_df, model, fourier_model, resid_model, r2, box_cox_params = \
                                               new_train_linear(reg, parameters[clst_num], f[reg], regularizer=linear_l1)
        elif regressor == "xgboost":
            if xgboost_params == None:
                train_df, model, fourier_model, resid_model, r2, box_cox_params  = \
                                               new_train_xgboost(reg, parameters[clst_num], f[reg])
            else:
                train_df, model, fourier_model, resid_model, r2, box_cox_params  = \
                                               new_train_xgboost(reg, parameters[clst_num], f[reg], xgboost_params=xgboost_params)
        
        predicted_df = new_predict(reg, parameters[clst_num], train_df, model, fourier_model, resid_model, 
                                   end_t_prediction='2016-05-07T23:00:00.000000000', features=f[reg], box_cox_params=box_cox_params)
        r2_train[reg] = r2
        r2_test[reg] = r2_score(verify_taxi.loc[: end_period][region_map[reg]].values, 
                                predicted_df.loc['2016-05-01T00:00:00.000000000': end_period]['taxi_call_num'].values)
    return r2_train, r2_test     

In [28]:
def new_predict(region, params, train_df, linear_model, fourier_model, resid_model, end_t_prediction, features, box_cox_params):
    '''new_feature_constructor(region, params, dataframe=taxi, weather=test_weather):
       params : ((period, component_num), ...) '''
    
    predict_time = pd.date_range('2016 May 1 00:00:00', end_t_prediction, freq = 'h')
    
    start = train_df.shape[0]
    fourier_time = np.arange(0, start + predict_time.shape[0])
    '''2016 May 1 00:00:00'''     '''2014-05-01 00:00:00'''
    
    for idx, h in enumerate(predict_time):
        #print(h, end=" ")
        train_df.loc[h] = [0]*train_df.shape[1]
        X = np.array([fun(2*np.pi*w*fourier_time[start+idx]/component[0]) for component in params[0] 
                                                       for w in range(1, component[1]+1) 
                                                       for fun in (np.sin, np.cos)]) 
        #coefs = np.array([fourier_model.params[coef] for coef in fourier_model.params.index])
        #X = np.hstack((np.array([1]), X))
        #train_df.ix[h, "fourier"] = coefs.dot(X)
        train_df.ix[h, "fourier"] = fourier_model.predict(X.reshape(1, -1))
        
        if resid_model != None:
            if box_cox_params == None:
                train_df.ix[h, "arima_residuals"] = resid_model.predict(start+idx, start+idx)
            else:
                ''' box_cox_params=(lmbda, min_value) '''
                resid_prediction = resid_model.predict(start+idx, start+idx)
                if resid_prediction == 0:
                    resid_prediction = 0.01
                train_df.ix[h, "arima_residuals"] = invboxcox(resid_prediction, box_cox_params[0]) - box_cox_params[1] - 0.1 
        else:
            train_df.ix[h, "arima_residuals"] = 0
        
        d = [("d_before", 24), ("two_d_before", 48), ("three_d_before", 72), ("four_d_before", 96), ("five_d_before", 120), 
             ("six_d_before", 144), ("one_h_before", 1), ("two_h_before", 2), ("three_h_before", 3), ("four_h_before", 4), 
             ("five_h_before", 5), ("six_h_before", 6), ("seven_h_before", 7), ("eight_h_before", 8), ("nine_h_before", 9), 
             ("ten_h_before", 10), ("eleven_h_before", 11), ("twelve_h_before", 12)]
        
        for name, value in d:
            train_df.ix[h, name] = train_df.loc[h - pd.Timedelta(hours=value)]['taxi_call_num']
                   
        dd = [("half_d_cumsum", (1, 12)), ("d_cumsum", (1, 24)), ("week_cumsum", (1, 168)), ("four_w_cumsum", (1, 168*4))]
            
        for name, value in dd:
            train_df.ix[h, name] = 0
            train_df[name] = train_df["taxi_call_num"].shift(1).rolling(min_periods=value[0], window=value[1]).sum()
                
        train_df["hour"] = train_df.index.hour
        train_df["week_day"] = train_df.index.dayofweek
        train_df["day"] = train_df.index.day
        train_df["month"] = train_df.index.month
        
        ''' take only single feature columns '''
        multicross_coll = itertools.combinations(train_df.columns[1 : 29].values, 2)
        for pair in multicross_coll:
            train_df.ix[h, pair[0] + "_" + pair[1]] = train_df.ix[h, pair[0]] * train_df.ix[h, pair[1]] 
            
        train_df["weekend"] = [1 if (train_df.weekday == 5 or train_df.weekday == 6) else 0 for train_df in train_df.index]
        #train_df = pd.concat([train_df, prediction_weather.ix[h, [0, 1, 3]]], axis=1)
        train_df.ix[h, ["daily_temp", "windchill", "events"]] = prediction_weather.ix[h, ["daily_temp", "windchill", "events"]]
        train_df = train_df.fillna(0.1)    
        
        p = linear_model.predict((train_df.loc[h][features].values).reshape(1, -1))
        #p_ = prediction(model, data.loc[h][features])
        
        train_df.ix[h, 'taxi_call_num'] = p
    
    print("yes")
    return train_df

##### try new approach on one region

In [25]:
%%time
dt, _, _ =  new_feature_constructor(region=1, params=parameters[5], dataframe=taxi, weather=test_weather)
all_features = dt.columns[1:].values

train_df, linear_model, fourier_model, resid_model, r2_sc = new_train_linear(region=76, params=parameters[2], 
                                                                             features_regions=all_features, regularizer=None)
#'2016 May 7 23:00:00'
predicted_df = new_predict(region=76, params=parameters[2], train_df=train_df, linear_model=linear_model, 
                           fourier_model=fourier_model, resid_model=resid_model, 
                           end_t_prediction='2016-05-07T23:00:00.000000000', features=all_features)
print("train r2: ", r2_sc)
print("test r2: ", r2_score(verify_taxi.loc[: '2016-05-07T23:00:00.000000000'][region_map[76]].values, 
                        predicted_df.loc['2016-05-01T00:00:00.000000000': '2016-05-07T23:00:00.000000000']['taxi_call_num'].values))

##### make feature selection for new feature set and write selected names into file 

In [29]:
def new_find_best_features(regions, clust_num):
    ''' find best features using xgboost. for each region take best features with cumsum 99%  '''
    features_good_regions = {}
    for reg in regions:
        print("REGION: ", reg, end=" ")
        model = XGBRegressor()
        region, region_fourier_model, region_resid_model, _ = new_feature_constructor(reg, 
                                        params=parameters[clust_num], dataframe=taxi, weather=test_weather)
        model.fit(region.iloc[:, 1:], region.taxi_call_num.values)

        significance_list = list(filter(lambda x: x[1] > 0, 
                                        sorted(zip(region.columns[1:], model.feature_importances_), 
                                               key=lambda x: x[1], reverse=True)))
        num_of_features = np.argwhere(np.cumsum([x[1] for x in significance_list]) > 0.98)[0][0]
        features_good_regions[reg] = [x[0] for x in significance_list][:num_of_features]
    return features_good_regions

In [29]:
arima_features = {}

In [3]:
for key in clusters.keys():
    for reg in clusters[key]:
        if reg not in arima_features.keys():
            print(reg, end="  ")
            features = new_find_best_features(regions=[reg, ], clust_num=key)
            arima_features.update(features)   

In [30]:
#with open("./data/best_arima_features.pickle", 'wb') as handle:
#    pickle.dump(arima_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
with open("./data/best_arima_features.pickle", 'rb') as handle:
    arima_features = pickle.load(handle)

In [76]:
arima_features.keys()

##### calculate r2 on train and tewt data using linear model 

In [45]:
r2_train_linear, r2_test_linear = {}, {}

In [141]:
r2_train_linear_arima = {0: 0.9119968410439852, 1: 0.9490993958448181, 2: 0.943737345988046, 3: 0.9036625074178369, 4: 0.9466296221816998, 5: 0.9588777967903648, 6: 0.960054348260383, 7: 0.960153827677181, 8: 0.9630107168485411, 9: 0.9196128547347804, 10: 0.9108417567173092, 11: 0.6631348607177621, 12: 0.8410582252645968, 13: 0.8295419250732765, 14: 0.7302400411528129, 15: 0.7834402657079477, 16: 0.9714554519015575, 17: 0.9605842497033593, 18: 0.9648910436591203, 19: 0.9577621312949011, 20: 0.9405281938425278, 21: 0.9380386528963218, 22: 0.7910002267377363, 23: 0.8515612301744745, 24: 0.7276042628518571, 25: 0.8191640848049861, 26: 0.7161004044715472, 27: 0.7495671422830372, 28: 0.5669772011181722, 29: 0.9684348086413028, 31: 0.9556701048009825, 32: 0.9650133517647733, 33: 0.9649441387634183, 34: 0.9594383309989786, 35: 0.9610764244230865, 36: 0.9551475323913264, 37: 0.9403064589191927, 38: 0.7922725193132002, 39: 0.690646267633203, 40: 0.7818518971498303, 41: 0.7040056545022533, 42: 0.8436528748367988, 43: 0.942073734917632, 44: 0.9687626757158736, 46: 0.9590823725537673, 47: 0.9220293942174321, 48: 0.9405209574013116, 49: 0.946652029292172, 50: 0.9035307786856728, 51: 0.9331990915030758, 52: 0.9146776816261705, 53: 0.919805101900297, 54: 0.9646747590363213, 55: 0.9751880738009553, 56: 0.9754434333226345, 57: 0.949680846355467, 58: 0.9189927353451454, 59: 0.8685822850103975, 60: 0.8381487363459071, 61: 0.5384164226603467, 62: 0.9099387848107289, 64: 0.8224134932086855, 65: 0.6022969917491963, 66: 0.871089830679304, 67: 0.9710480280444334, 68: 0.9508173874702501, 69: 0.9598122071651816, 70: 0.7763460810762226, 71: 0.7630640798866706, 72: 0.6576823561295768, 73: 0.5810609466466641, 74: 0.6554772619179164, 75: 0.8392609368012084, 76: 0.44377299781263513, 77: 0.964514103071619, 78: 0.7830936527794072, 79: 0.5139144199916669, 80: 0.7330176796324457, 81: 0.742519273475974, 82: 0.6851359599854614, 83: 0.4909995505165408, 84: 0.4147587789052622, 85: 0.5935544860672606, 86: 0.6740350633264847, 87: 0.7619819199147659, 88: 0.7990076159676912, 89: 0.7203847535830064, 90: 0.740876688354576, 91: 0.7012984023194486, 93: 0.664839890191916, 94: 0.6676659483673352, 95: 0.8888979120188998, 96: 0.8914649223476513, 97: 0.8744235165659653, 98: 0.6059303633386529, 99: 0.8647048187515344, 100: 0.7421851735152822, 101: 0.7312302424391292}
r2_test_linear_arima = {0: 0.8224467270018745, 1: 0.8975756214065198, 2: 0.8294046862666157, 3: 0.7663838064239681, 4: 0.8431752784914119, 5: 0.9011922926114393, 6: 0.8774600199308159, 7: 0.8846754875845705, 8: 0.8697286000023449, 9: 0.8427925374129068, 10: 0.49706682574593075, 11: 0.34039779471319975, 12: 0.7255302029997159, 13: 0.7587106261854359, 14: 0.3547599264018195, 15: 0.7222033470891763, 16: 0.8345972024242171, 17: 0.7390757491030218, 18: 0.880570944091634, 19: 0.7976726311293122, 20: 0.8683113100957064, 21: 0.2880637534265442, 22: 0.5962546472555386, 23: 0.7411952534252351, 24: 0.5363678531125227, 25: 0.7164521407490696, 26: 0.6559851659951002, 27: 0.6614637890856322, 28: 0.2671194907303083, 29: 0.8580378716072333, 31: 0.6749600544509686, 32: 0.9257704058538752, 33: 0.9081586143848964, 34: 0.927492574751153, 35: 0.9502439721796937, 36: 0.31994412786118265, 37: 0.8536671267686329, 38: 0.6645958006091831, 39: 0.5673199613405476, 40: 0.633976185293541, 41: 0.5688636989071445, 42: 0.7069484435625198, 43: 0.894433844095866, 44: 0.9077846412952563, 46: 0.9241870948493272, 47: 0.9201632287308147, 48: 0.8348791521685238, 49: 0.9382986463946186, 50: 0.84936762793988, 51: 0.7317875899734994, 52: -0.03312705417382422, 53: 0.8915816793837884, 54: 0.9003866065813593, 55: 0.9640399455276345, 56: 0.9619554479153883, 57: 0.7306935523230349, 58: 0.9361160175125954, 59: 0.8369987620985395, 60: 0.716976387901631, 61: 0.390748495521107, 62: 0.858646030507306, 64: 0.34828170542399695, 65: 0.46603778547549723, 66: 0.8092417588765947, 67: 0.9543767449083165, 68: 0.8797398323477555, 69: 0.8281574089182707, 70: 0.6648113074243733, 71: 0.6982245802062891, 72: 0.5077709610738068, 73: 0.3757125660700622, 74: 0.5133585096421143, 75: 0.8009431635222025, 76: 0.26938407108995677, 77: 0.3775437523890195, 78: 0.4131924021461324, 79: 0.40689657715197936, 80: 0.5919014199575632, 81: 0.600791292887356, 82: 0.6460620668928954, 83: 0.4116101962749471, 84: 0.21853389637345721, 85: 0.5545338531824254, 86: 0.5395077098279726, 87: 0.6166536923668474, 88: 0.6147714171466506, 89: 0.3307060563363301, 90: 0.4075719573698814, 91: 0.6590552758294967, 93: 0.5879662859012847, 94: 0.4239159452929804, 95: 0.8841448557966239, 96: 0.8102930416943861, 97: 0.8291006834158748, 98: 0.4044392026041336, 99: 0.7421790689602673, 100: 0.7308013928086403, 101: 0.6084488146258936}

In [80]:
r2_train_linear_arima.keys()

In [83]:
for key in clusters.keys():
    for reg in clusters[key]:
        if (reg in arima_features.keys()) and (reg not in r2_train_linear_arima.keys()): 
            try:
                r2_train, r2_test = new_get_r2(reg_lst=[reg, ], clst_num=key, end_period='2016-05-07T23:00:00.000000000', 
                               f=arima_features, regressor="linear", linear_l1=None, xgboost_params=None)
                r2_train_linear_arima.update(r2_train)
                r2_test_linear_arima.update(r2_test) 
            except LinAlgError as e:
                print(e)
                continue    

In [87]:
print(r2_test_linear_arima)

##### assign r2 to 0 value to regions for which it was not possible to calculate the forecast due to an error in the singularity of the matrix 

In [142]:
for idx in range(102):
    if idx not in r2_train_linear_arima.keys():
        r2_train_linear_arima[idx], r2_test_linear_arima[idx] = 0, 0        

In [99]:
plot_bars_1(range(102), list(zip(r2_train, r2_train_combination, [r2_train_linear[key] for key in sorted(r2_train_linear.keys(), reverse=False)])), 
            "r2 train data for three cases: NO arima + NO feature combinations; NO arima + feature combinations; arima + feature combinations", 
                        ["simple", "simple + feature combinations", "arima + feature combinations"])
#double click at image to make it larger

In [100]:
plot_bars_1(range(102), list(zip(r2_test, r2_test_combination, [r2_test_linear[key] for key in sorted(r2_test_linear.keys(), reverse=False)])), 
            "r2 test data for three cases: NO arima + NO feature combinations; NO arima + feature combinations; arima + feature combinations", 
                        ["simple", "simple + feature combinations", "arima + feature combinations"])
#double click at image to make it larger

##### in most cases, the arima improved the result in case of ElasticNet regressor 

##### do the same for xgbregressor

In [32]:
r2_train_xgb_arima, r2_test_xgb_arima = {}, {}

In [135]:
r2_train_xgb_arima={0: 0.9214920053766192, 1: 0.9565051139872823, 2: 0.9524984535871854, 3: 0.9161126058339943, 4: 0.9532701729255865, 5: 0.962957385000979, 6: 0.9650162643308293, 7: 0.9656536509878771, 8: 0.9691180666633495, 9: 0.9278118395678898, 10: 0.9372518053421589, 11: 0.7289316087722043, 12: 0.8640998453988998, 13: 0.8576359716009263, 14: 0.773876438274451, 15: 0.8131249519361132, 16: 0.9797346778931081, 17: 0.9693129461858269, 18: 0.9686760492846317, 19: 0.9702138345787072, 20: 0.950167373733363, 21: 0.9460265106395507, 22: 0.8322885558421135, 23: 0.8732464576420843, 24: 0.8073884221638478, 25: 0.8683004040628838, 26: 0.7621688434248854, 27: 0.780295363627681, 28: 0.664194744558792, 29: 0.9787133390870407, 31: 0.9627559802017741, 32: 0.970261888508915, 33: 0.9688808171918075, 34: 0.9631522862192884, 35: 0.9650366602578905, 36: 0.9612684838587683, 37: 0.9507960963670328, 38: 0.8381791325607305, 39: 0.7709458283681332, 40: 0.8210844700298652, 41: 0.7475885222062886, 42: 0.8659285252626733, 43: 0.9522031432162338, 44: 0.9719298419407584, 45: 0.9793251879018998, 46: 0.9642964895928486, 47: 0.9341391258950772, 48: 0.9522953698660679, 49: 0.955652282864946, 50: 0.9207764455352855, 51: 0.9485564103254078, 52: 0.9363445711790026, 53: 0.9337301872579058, 54: 0.9711758776015231, 56: 0.9790052959720803, 57: 0.9576809561652376, 58: 0.9333825326927186, 59: 0.8862909565106278, 60: 0.8579073076159196, 61: 0.6032896005180468, 62: 0.9372823425798573, 64: 0.8735511989587517, 65: 0.670509012797903, 66: 0.8892927135427284, 67: 0.9758991798683403, 68: 0.9594567015269208, 69: 0.966300068748117, 70: 0.8083413435096367, 71: 0.8019550545893044, 72: 0.7015996052973389, 73: 0.6356038536307334, 74: 0.7115121002214826, 75: 0.8922117864931882, 76: 0.5319385241315914, 77: 0.9741485785443685, 78: 0.8130188112463974, 79: 0.5976237101613325, 80: 0.7667965110409082, 81: 0.7914002503876606, 82: 0.7355116402720274, 83: 0.589853856324517, 84: 0.5090752709257473, 85: 0.663702779394391, 86: 0.7679961361001919, 87: 0.7972013425617779, 88: 0.842946521419968, 89: 0.799019427072988, 90: 0.8115985204344685, 91: 0.7663757911362521, 93: 0.7068528280381172, 94: 0.7286820853815528, 95: 0.9021616192184578, 96: 0.9039269018488986, 97: 0.8856357345322903, 98: 0.6605872101015378, 99: 0.8728223589445503, 100: 0.7651062406186728, 101: 0.7569002117525757}
r2_test_xgb_arima={0: 0.8481615838647825, 1: 0.905808697123601, 2: 0.8166630540018108, 3: 0.7838564658271712, 4: 0.830035266708227, 5: 0.904965399715668, 6: 0.9204841859383951, 7: 0.9070372488632373, 8: 0.8247266852228292, 9: 0.8509253030116875, 10: 0.7187072084214121, 11: 0.40449090520637576, 12: 0.711454770457467, 13: 0.7390267006042794, 14: 0.6040863927055766, 15: 0.8001651019381283, 16: 0.785013220666562, 17: 0.8500841796535683, 18: 0.8790481022214711, 19: 0.9254226791382206, 20: 0.8580084962273047, 21: 0.8430381848060818, 22: 0.6119726274502911, 23: 0.7569432268054468, 24: 0.3296640884227713, 25: 0.6698718130627779, 26: 0.6852250193318408, 27: 0.6899049779340141, 28: 0.3421338998289837, 29: 0.8339248969045001, 31: 0.9089638899479211, 32: 0.90893597065934, 33: 0.9078595608888108, 34: 0.9234537647796213, 35: 0.9387989232334488, 36: 0.9410054769793917, 37: 0.8825659273148379, 38: 0.6644260443728575, 39: 0.6285325378477609, 40: 0.6452317733271911, 41: 0.6420420068445332, 42: 0.6789572218363911, 43: 0.8984116068162411, 44: 0.9074334351323237, 45: 0.9495224627674613, 46: 0.927152883307969, 47: 0.9237435050807363, 48: 0.8776136393798739, 49: 0.9338972969234932, 50: 0.8732014232297314, 51: 0.7402906528252963, 52: 0.26261783006173867, 53: 0.9051177479645269, 54: 0.9093933374115316, 56: 0.966275493890232, 57: 0.7384871406275008, 58: 0.9480765807747569, 59: 0.8014240980383491, 60: 0.7149754070990242, 61: 0.4995477138039067, 62: 0.8251047217335167, 64: 0.2939359550676621, 65: 0.39441320273291003, 66: 0.8341110095068748, 67: 0.9553225001569028, 68: 0.8977085747414459, 69: 0.8018873144182823, 70: 0.6825180205687982, 71: 0.6837494922326253, 72: 0.5205884728256239, 73: 0.39489897448738454, 74: 0.5001982699118461, 75: 0.7509110704610313, 76: -0.13720105973938979, 77: 0.5188345480335187, 78: 0.41371502464746945, 79: 0.4716648899747182, 80: 0.6330714867135665, 81: 0.6002158943011836, 82: 0.6946024056346632, 83: 0.48847072767957256, 84: 0.2884008944201022, 85: 0.5830002269123122, 86: 0.530856697937132, 87: 0.729740383799647, 88: 0.794178353562284, 89: 0.32054943075318143, 90: 0.21859049602791403, 91: 0.7026124050656959, 93: 0.6046367788268393, 94: 0.4784931117620249, 95: 0.8945185488562153, 96: 0.8668443785046096, 97: 0.8306672297735264, 98: 0.4705448174755842, 99: 0.8276834300391533, 100: 0.7238379985802252, 101: 0.634600923512749}

In [104]:
#for key in clusters.keys():
for key in clusters.keys():
    for reg in clusters[key]:
        if (reg in arima_features.keys()) and (reg not in r2_train_xgb_arima.keys()): 
            try:
                r2_train, r2_test = new_get_r2(reg_lst=[reg, ], clst_num=key, end_period='2016-05-07T23:00:00.000000000', 
                               f=arima_features, regressor="xgboost", linear_l1=None, xgboost_params=None)
                r2_train_xgb_arima.update(r2_train)
                r2_test_xgb_arima.update(r2_test) 
            except LinAlgError as e:
                print(e)
                continue

In [139]:
for idx in range(102):
    if idx not in r2_train_xgb_arima.keys():
        r2_train_xgb_arima[idx], r2_test_xgb_arima[idx] = 0, 0     

In [112]:
plot_bars_1(range(102), list(zip(xgb_r2_train,_arima_arima [r2_train_xgb_arima[key] for key in sorted(r2_train_xgb_arima.keys(), reverse=False)])), 
            "TRAIN NO arima + xgb feature combinations; arima + xgb feature combinations", ["xgb", "arima + xgb"])
#double click at image to make it larger

In [113]:
plot_bars_1(range(102), list(zip(xgb_r2_test, [r2_test_xgb_arima[key] for key in sorted(r2_test_xgb_arima.keys(), reverse=False)])), 
            "TEST NO arima + xgb feature combinations; arima + xgb feature combinations", ["xgb", "arima + xgb"])
#double click at image to make it larger

##### with rare exceptions, adding an arima prediction did not improve final result

##### collect all results together and plot them at one histogram <br> here we plot r2 score for test dataset: <br>linear regression, linear regression with parameters combination, <br> linear regression with parameters combination and ARIMA, xgboost and feature combination used all features without selection; <br> the same but with feature selection; <br> same with feature selection and ARIMA prediction 

In [144]:
lin_arima = [r2_test_xgb_arima[key] for key in sorted(r2_test_xgb_arima.keys(), reverse=False)]
xgb_arima = [r2_test_xgb_arima[key] for key in sorted(r2_test_xgb_arima.keys(), reverse=False)]

In [152]:
plot_bars_1(range(102), list(zip(r2_test, r2_test_combination, lin_arima, xgb_r2_test, xgb_r2_test_select, xgb_arima)), 
            "TEST all variants", ["linear", "linear+comb", "linear+comb+arima", "xgb+comb", "xgb+comb+best_params", "xgb+comb+arima"])
#double click at image to make it larger

#### Analyzing the results it is evident that in most cases the best result is shown by xgbregressor with features combination and selection. Adding ARIMA prediction infortunately do not improve results on regions with bad prediction quality. In some cases other regression models show better result 

##### in most cases best regression model - xgbregressor with unified fourier components without ARIMA.  

##### from histogram select model that gives the best result, individually for each region 

In [32]:
best_model = {"xgb_comb_bestparams": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 26, 27, 28, 29, 
                          30, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 
                          58, 59, 60, 61, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 81, 82, 83, 84, 88, 90, 
                          91, 92, 93, 95, 96, 97, 98, 99, 100, 101],
             "xgb_comb": [39, 40, 52, 80, 85, 87, 89], 
             "linear_combination": [24, 64, 75], 
             "linear_combination_arima": [10, 25, 62, 86, 94]}

In [33]:
p={'learning_rate':0.1, 'n_estimators':400, 'max_depth':7, 'min_child_weight':5, 'gamma':0.1, 'subsample':0.9, 
            'colsample_bytree':0.6, 'objective':'reg:linear', 'scale_pos_weight':1, 'seed':27, 'reg_alpha':0}

In [34]:
dict_features = dict(zip(range(102), features))

In [35]:
def plot_prediction(from_t, to_t, title, figsize=(25, 10)):
    for key in clusters.keys():
        for reg in clusters[key]:
            print("REGION: ", reg)
            model = None
            for m, regions in best_model.items():
                if reg in regions:
                    model = m
                    break
            try:
                if model == "linear_combination_arima":
                    train_df, linear_model, fourier_model, resid_model, r2_sc, box_cox_params = new_train_linear(region=reg, params=parameters[key], 
                                                                                 features_regions=arima_features[reg], regularizer=None)
                    predicted_df = new_predict(reg, parameters[key], train_df, linear_model, fourier_model, resid_model, 
                                       end_t_prediction=to_t, features=arima_features[reg], box_cox_params=box_cox_params)
                    print("Linear arima r2 train score: ", r2_sc)
                    
                elif model == "linear_combination":
                    mod, r2 = train_linear(region=reg, features_regions=dict_features[reg], regularizer=None)
                    predicted_df = predict(region=reg, model=mod, end_t_prediction=to_t, features=dict_features[reg])
                    print("Linear r2 train score: ", r2)

                else:
                    data, _ = feature_constructor(region=reg, regres=((168, 70), (8760, 20)))
                    if model == "xgb_comb_bestparams":
                        mod = XGBRegressor(**p)
                    elif model == "xgb_comb":
                        mod = XGBRegressor()
                    mod.fit(data[dict_features[reg]].values, data.taxi_call_num.values)
                    print("XGB train r2: ", mod.score(data[dict_features[reg]].values, data.taxi_call_num.values))
                    predicted_df = predict(region=reg, model=mod, end_t_prediction=to_t, features=dict_features[reg])

                r2_test = r2_score(verify_taxi.loc[from_t: to_t][region_map[reg]].values, 
                                    predicted_df.loc[from_t: to_t]['taxi_call_num'].values)
                print("Linear r2 test score: ", r2_test)

                plt.figure(figsize=figsize)
                plt.plot(verify_taxi.loc[from_t: to_t].index.values, 
                         verify_taxi.loc[from_t: to_t][region_map[reg]].values,
                         color="red", alpha = 0.5, label='real value')
                plt.plot(verify_taxi.loc[from_t: to_t].index.values, 
                         predicted_df.loc[from_t: to_t]['taxi_call_num'].values, 
                         color="green", alpha = 0.7, label='predicted')
                plt.xlabel('time (hour)')
                plt.title(title + str(r2_test))
                plt.legend()
                plt.grid()
                plt.show()

            except LinAlgError as e:
                print(e)
                continue    

In [49]:
plot_prediction(from_t='2016-05-01T00:00:00.000000000', to_t='2016-05-30T23:00:00.000000000', title="prediction for may 2016; r2: ", 
                figsize=(80, 10))

##### make prediction and save dataframe to csv file

In [36]:
r_pred, c_red = verify_time.shape[0] + time.shape[0], data_2month.T[0, :].shape[0]

t_full = pd.date_range('2014 May 1 00:00:00', periods = data_2year.shape[1] +  data_2month.shape[1] - 2, freq = 'h')
f = lambda x: pd.DataFrame(np.zeros((r_pred, c_red)), index = [t_full], columns=[data_2month.T[0, :].astype(int)])

prediction_df = f(1)

In [37]:
prediction_df.head()

In [38]:
def make_prediction(from_t, to_t, df_save):
    t_pred = pd.date_range('2014-05-01 00:00:00', to_t, freq = 'h')
    for key in clusters.keys():
        for reg in clusters[key]:
            print("REGION: ", reg, end=" ")
            model = None
            for m, regions in best_model.items():
                if reg in regions:
                    model = m
                    break
            try:
                if model == "linear_combination_arima":
                    train_df, linear_model, fourier_model, resid_model, r2_sc, box_cox_params = new_train_linear(region=reg, params=parameters[key], 
                                                                                 features_regions=arima_features[reg], regularizer=None)
                    predicted_df = new_predict(reg, parameters[key], train_df, linear_model, fourier_model, resid_model, 
                                       end_t_prediction=to_t, features=arima_features[reg], box_cox_params=box_cox_params)
                    df_save.iloc[:t_pred.shape[0], reg] = predicted_df['taxi_call_num'].values
                    
                elif model == "linear_combination":
                    mod, r2 = train_linear(region=reg, features_regions=dict_features[reg], regularizer=None)
                    predicted_df = predict(region=reg, model=mod, end_t_prediction=to_t, features=dict_features[reg])
                    df_save.iloc[:t_pred.shape[0], reg] = predicted_df['taxi_call_num'].values

                else:
                    data, _ = feature_constructor(region=reg, regres=((168, 70), (8760, 20)))
                    if model == "xgb_comb_bestparams":
                        mod = XGBRegressor(**p)
                    elif model == "xgb_comb":
                        mod = XGBRegressor()
                    mod.fit(data[dict_features[reg]].values, data.taxi_call_num.values)
                    predicted_df = predict(region=reg, model=mod, end_t_prediction=to_t, features=dict_features[reg])
                    df_save.iloc[:t_pred.shape[0], reg] = predicted_df['taxi_call_num'].values

                r2_test = r2_score(verify_taxi.loc[from_t: to_t][region_map[reg]].values, 
                                    predicted_df.loc[from_t: to_t]['taxi_call_num'].values)
                print("Linear r2 test score: ", r2_test)


            except LinAlgError as e:
                print(e)
                continue    

##### make prediction for may and first week of june 2016

In [43]:
make_prediction(from_t='2016-05-01T00:00:00.000000000', to_t='2016-06-07T23:00:00.000000000', df_save=prediction_df)

In [49]:
prediction_df.loc['2016-05-01T00:00:00':].shape

##### select only prediction time - may june 2016

In [51]:
prediction_df.loc['2016-05-01T00:00:00':].round().to_csv("./data/week_5_prediction", sep='\t', columns=list(prediction_df.columns))

In [52]:
k = lambda name: pd.read_csv(name, sep='\t', index_col="Unnamed: 0")

prediction_df = k("./data/week_5_prediction")

In [53]:
prediction_df.head()

In [1]:
''' take exact features '''
#np.take(dt.columns[1:], [*np.argwhere(features_RFE[1] == True).flatten()])